In [8]:
from keras import Input, Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Dropout, Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

from tiny.util import attach_device_train_label, replace_invalid_filename_char, get_stable_feature
from utils_.util_log import *
tmp = get_stable_feature('1002')
tmp.head()

2018-10-04 20:20:54,651 util_log.py[33] INFO Begin to run get_stable_feature with:['1002'], {}
2018-10-04 20:20:54,663 util_cache_file.py[81] DEBUG fn:get_stable_feature, para:['1002'], kw:{}
2018-10-04 20:20:54,665 util_cache_file.py[21] DEBUG try to read cache from file:./cache/get_stable_feature_['1002']_{}.csv, type:csv
2018-10-04 20:21:19,078 util_cache_file.py[35] DEBUG Return 72727 resut from file cache:./cache/get_stable_feature_['1002']_{}.csv
2018-10-04 20:21:19,082 util_log.py[38] INFO cost:  24.43 sec: ==='get_stable_feature' end (['1002'], {}) 


,device,lda_app_package_None_drop:0_0,lda_app_package_None_drop:0_1,lda_app_package_None_drop:0_2,lda_app_package_None_drop:0_3,lda_app_package_None_drop:0_4,lda_app_package_None_drop:0_app_length,lda_usage_package_count_drop:0_0,lda_usage_package_count_drop:0_1,lda_usage_package_count_drop:0_2,lda_usage_package_count_drop:0_3,lda_usage_package_count_drop:0_4,lda_usage_package_count_drop:0_app_length,lda_usage_package_sum_drop:0_0,lda_usage_package_sum_drop:0_1,lda_usage_package_sum_drop:0_2,lda_usage_package_sum_drop:0_3,lda_usage_package_sum_drop:0_4,lda_usage_package_sum_drop:0_app_length,lda_app_package_None_drop:18363_0,lda_app_package_None_drop:18363_1,lda_app_package_None_drop:18363_2,lda_app_package_None_drop:18363_3,lda_app_package_None_drop:18363_4,lda_app_package_None_drop:18363_app_length,lda_usage_package_count_drop:18363_0,lda_usage_package_count_drop:18363_1,lda_usage_package_count_drop:18363_2,lda_usage_package_count_drop:18363_3,lda_usage_package_count_drop:18363_4,lda_usage_package_count_drop:18363_app_length,lda_usage_package_sum_drop:18363_0,lda_usage_package_sum_drop:18363_1,lda_usage_package_sum_drop:18363_2,lda_usage_package_sum_drop:18363_3,lda_usage_package_sum_drop:18363_4,lda_usage_package_sum_drop:18363_app_length,lda_usage_p_sub_type_count_drop:18363_0,lda_usage_p_sub_type_count_drop:18363_1,lda_usage_p_sub_type_count_drop:18363_2,lda_usage_p_sub_type_count_drop:18363_3,lda_usage_p_sub_type_count_drop:18363_4,lda_usage_p_sub_type_count_drop:18363_app_length,lda_usage_p_sub_type_count_drop:0_0,lda_usage_p_sub_type_count_drop:0_1,lda_usage_p_sub_type_count_drop:0_2,lda_usage_p_sub_type_count_drop:0_3,lda_usage_p_sub_type_count_drop:0_4,lda_usage_p_sub_type_count_drop:0_app_length,lda_usage_p_sub_type_sum_drop:18363_0,lda_usage_p_sub_type_sum_drop:18363_1,lda_usage_p_sub_type_sum_drop:18363_2,lda_usage_p_sub_type_sum_drop:18363_3,lda_usage_p_sub_type_sum_drop:18363_4,lda_usage_p_sub_type_sum_drop:18363_app_length,lda_usage_p_sub_type_sum_drop:0_0,lda_usage_p_sub_type_sum_drop:0_1,lda_usage_p_sub_type_sum_drop:0_2,lda_usage_p_sub_type_sum_drop:0_3,lda_usage_p_sub_type_sum_drop:0_4,lda_usage_p_sub_type_sum_drop:0_app_length,package_0,package_1,package_2,package_3,package_4,package_5,package_6,duration_0,duration_1,duration_2,duration_3,duration_4,duration_5,duration_6,package_wk_0,package_wk_1,duration_wk_0,duration_wk_1,action_wk_0,action_wk_1,pkg_nunique,pkg_count,dur_sum,start_base_nunique,wk_compare_app_count,wk_comapre_dur,wk_compare_action_count,action_daily,dur_sum_daily,sum_pkg_nunique,sum_day_nunique,sum_span24_0_sum_p,sum_span24_1_sum_p,sum_span24_2_sum_p,sum_span24_3_sum_p,sum_span24_4_sum_p,sum_span24_5_sum_p,sum_span24_6_sum_p,sum_span24_7_sum_p,sum_span24_8_sum_p,sum_span24_9_sum_p,sum_span24_10_sum_p,sum_span24_11_sum_p,sum_span24_12_sum_p,sum_span24_13_sum_p,sum_span24_14_sum_p,sum_span24_15_sum_p,sum_span24_16_sum_p,sum_span24_17_sum_p,sum_span24_18_sum_p,sum_span24_19_sum_p,sum_span24_20_sum_p,sum_span24_21_sum_p,sum_span24_22_sum_p,sum_span24_23_sum_p,sum_span24_0_count_p,sum_span24_1_count_p,sum_span24_2_count_p,sum_span24_3_count_p,sum_span24_4_count_p,sum_span24_5_count_p,sum_span24_6_count_p,sum_span24_7_count_p,sum_span24_8_count_p,sum_span24_9_count_p,sum_span24_10_count_p,sum_span24_11_count_p,sum_span24_12_count_p,sum_span24_13_count_p,sum_span24_14_count_p,sum_span24_15_count_p,sum_span24_16_count_p,sum_span24_17_count_p,sum_span24_18_count_p,sum_span24_19_count_p,sum_span24_20_count_p,sum_span24_21_count_p,sum_span24_22_count_p,sum_span24_23_count_p,p_type_knn_cnt_top#0,p_type_knn_cnt_top#1,p_type_knn_name_top#0,p_type_knn_name_top#1,p_type_knn_cnt_0_p,p_type_knn_cnt_1_p,p_type_knn_cnt_2_p,p_type_knn_cnt_3_p,p_type_knn_cnt_4_p,p_type_knn_cnt_6_p,p_type_knn_cnt_7_p,p_type_knn_cnt_8_p,p_type_knn_cnt_9_p,p_type_knn_cnt_10_p,p_type_knn_cnt_11_p,p_type_knn_cnt_12_p,p_type_knn_cnt_13_p,p_type_knn_cnt_14_p,p_type_knn_cnt_15_p,p_type_knn_cnt_16_p,p_type_knn_cnt_17_p,p_type_knn_cnt_1

In [13]:
logger.debug(f'{tmp.shape}, {tmp.shape}')

2018-10-04 20:27:39,636 <ipython-input-13-8929f52495aa>[1] DEBUG (72727, 1349), (72727, 1349)


In [12]:
logger.debug?

In [ ]:
best_epoch = np.array(history.history['val_loss']).argmin() + 1
best_score = np.array(history.history['val_loss']).min()

#pre_x = test.drop(['sex', 'age', 'sex_age', 'device'], axis=1)
sub = pd.DataFrame(model_load.predict_proba(test), columns=get_label_cat())


sub['DeviceID'] = test['device'].values
sub = sub[
    ['DeviceID', '1-0', '1-1', '1-2', '1-3', '1-4', '1-5', '1-6', '1-7', '1-8', '1-9', '1-10', '2-0', '2-1', '2-2',
     '2-3', '2-4', '2-5', '2-6', '2-7', '2-8', '2-9', '2-10']]

file = f'./sub/ensemble_{best}_epoch_{best_epoch}.csv'
file = replace_invalid_filename_char(file)
logger.info(f'sub file save to {file}')
sub = round(sub, 10)
sub.to_csv(file, index=False)



In [14]:
print( np.unique(y_train) )
print( np.unique(y_test) )

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [16]:

gbm = XGBClassifier(
                objective='multi:softprob',
                eval_metric='mlogloss',
                num_class=22,
                n_estimators=20,
                max_depth=3,

                min_child_weight=1,
                learning_rate=0.1,

                silent=True,
                gamma=0,
                max_delta_step=0,
                subsample=1,
                colsample_bytree=1,
                colsample_bylevel=1,
                reg_alpha=1,
                reg_lambda=1,
                scale_pos_weight=1,
                seed=1,
                missing=None)
# print(random_search.grid_scores_)
gbm.fit(X_train, y_train,  eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=True )

results = gbm.evals_result()

print(results)

[0]	validation_0-mlogloss:3.05177
Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-mlogloss:3.01924
[2]	validation_0-mlogloss:2.99001
[3]	validation_0-mlogloss:2.96482
[4]	validation_0-mlogloss:2.94357
[5]	validation_0-mlogloss:2.92285
[6]	validation_0-mlogloss:2.90434
[7]	validation_0-mlogloss:2.88765
[8]	validation_0-mlogloss:2.87236
[9]	validation_0-mlogloss:2.8587
[10]	validation_0-mlogloss:2.84676
[11]	validation_0-mlogloss:2.8353
[12]	validation_0-mlogloss:2.82478
[13]	validation_0-mlogloss:2.81524
[14]	validation_0-mlogloss:2.80597
[15]	validation_0-mlogloss:2.79737
[16]	validation_0-mlogloss:2.79012
[17]	validation_0-mlogloss:2.78346
[18]	validation_0-mlogloss:2.77655
[19]	validation_0-mlogloss:2.77088
{'validation_0': {'mlogloss': [3.051766, 3.019241, 2.990009, 2.964824, 2.943568, 2.922853, 2.904342, 2.887646, 2.872357, 2.858696, 2.846756, 2.835304, 2.824777, 2.815245, 2.805973, 2.797369, 2.790115, 2.783456, 2.776547, 2.770876]}}


[3.051766,
 3.019241,
 2.990009,
 2.964824,
 2.943568,
 2.922853,
 2.904342,
 2.887646,
 2.872357,
 2.858696,
 2.846756,
 2.835304,
 2.824777,
 2.815245,
 2.805973,
 2.797369,
 2.790115,
 2.783456,
 2.776547,
 2.770876]

In [19]:
best_epoch = np.array(results['validation_0']['mlogloss']).argmin()+1
best_score = np.array(results['validation_0']['mlogloss']).min()

print(f'{best_epoch}_{best_score}')


20_2.770876
